In [1]:
from openai import OpenAI
from typing import List, Dict

def ask(messages: List[Dict]):
        client = OpenAI(api_key='sk-a6bb624707354861abff16885ffe5252', base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
        response = client.chat.completions.create(
                model = 'qwen-plus',
                temperature = 0.3,
                messages = messages
        )

        return response

<h2>一个英译中对话的例子，多次交互实现Self-Refine效果<h2

In [2]:
# reference <https://github.com/andrewyng/translation-agent>

source_text = """
Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they should be applied to AI applications, not to general-purpose AI technology.
"""

<h2>第一轮交互，给llm设定成翻译家角色，进行翻译任务<h2

In [9]:
# 构建第一轮对话的system prompt和user prompt
first_translate_system_message = f"You are an expert linguist, specializing in translation from English to Simplified Chinese."

first_user_translation_prompt = f"""This is an English to Chinese translation, please provide the Simplified Chinese translation for this text. \\
Do not provide any explanations or text apart from the translation.
English: {source_text}

Chinese:"""

## 将 source text 作为参数传入，构造好完整的输入
first_user_prompt = first_user_translation_prompt.format(source_text=source_text)

print(first_user_prompt)

This is an English to Chinese translation, please provide the Simplified Chinese translation for this text. \
Do not provide any explanations or text apart from the translation.
English: 
Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they should be applied to AI applications, not to general-purpose AI technology.


Chinese:


In [10]:
messages = [
    {"role":"system", "content": first_translate_system_message},
    {"role":"user", "content": first_user_prompt}
]

first_response = ask(messages=messages)

translation_1 = first_response.choices[0].message.content
print(translation_1)

上周，我在美国国会的一个活动上谈到了人工智能和监管问题，该活动吸引了立法者和商界领袖参加。我对开源社区在抵制可能抑制创新的法规方面取得的进展感到鼓舞。但反对开源的人正在改变他们的论点，最新的担忧集中在开源对国家安全的影响上。我希望我们都能继续保护开源！

根据我与立法者的交谈，我对美国联邦政府在现实地理解人工智能风险方面取得的进展感到鼓舞。明确地说，需要设立一些规则。但这些规则应该应用于人工智能的具体应用，而不是通用的人工智能技术。


<h2>第二轮交互，给llm设定成评价角色，进行自我改进<h2

In [20]:
reflect_system_prompt = "You are an expert linguist specializing in translation from English to Simplified Chinese. You will be provided with a source text and its translation and your goal is to improve the translation."
# 不仅是语言专家，还要对翻译结果进行评价，目标是提升翻译质量
reflect_prompt = f"""Your task is to carefully read a source text and a translation from English to Simplified Chinese, and then give constructive criticism and helpful suggestions to improve the translation. \\

The source text and initial translation, delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT> and <TRANSLATION></TRANSLATION>, are as follows:

<SOURCE_TEXT>
{source_text}
</SOURCE_TEXT>

<TRANSLATION>
{translation_1}
</TRANSLATION>

When writing suggestions, pay attention to whether there are ways to improve the translation's \n\
(i) accuracy (by correcting errors of addition, mistranslation, omission, or untranslated text),\n\
(ii) fluency (by applying Simplified Chinese grammar, spelling and punctuation rules, and ensuring there are no unnecessary repetitions),\n\
(iii) style (by ensuring the translations reflect the style of the source text and takes into account any cultural context),\n\
(iv) terminology (by ensuring terminology use is consistent and reflects the source text domain; and by only ensuring you use equivalent idioms Simplified Chinese).\n\

Write a list of specific, helpful and constructive suggestions for improving the translation.
Each suggestion should address one specific part of the translation.
Output only the suggestions and nothing else."""
# <SOURCE_TEXT> 和 <TRANSLATION> 作为参数传入，构造好完整的输入

feedback_user_prompt = reflect_prompt.format(
        source_text=source_text,
        translation_1=translation_1,
    )
print(feedback_user_prompt)

Your task is to carefully read a source text and a translation from English to Simplified Chinese, and then give constructive criticism and helpful suggestions to improve the translation. \

The source text and initial translation, delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT> and <TRANSLATION></TRANSLATION>, are as follows:

<SOURCE_TEXT>

Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they sh

In [16]:
message2 = [
    {"role":"system", "content": reflect_system_prompt},
    {"role":"user", "content": feedback_user_prompt}
]

response = ask(messages=message2)

reflection = response.choices[0].message.content
print(reflection)

1. 在第一句中，“at an event that was attended by” 可以更自然地翻译为“在一个由...参加的活动上”，即：“上周，我在美国国会的一个由立法者和商界领袖参加的活动上谈到了人工智能和监管问题。”
2. 第二句中的“fending off regulations” 翻译为“抵制可能抑制创新的法规”可以更准确地表达原文的意思，建议改为：“我对开源社区在抵制那些可能抑制创新的法规方面取得的进展感到鼓舞。”
3. 第三句中的“opponents of open source are continuing to shift their arguments” 可以翻译得更加流畅，建议改为：“然而，反对开源的人士不断改变他们的论点，最近的担忧集中在开源对国家安全的影响上。”
4. 第四句中的“keep protecting open source!” 可以翻译得更加自然，建议改为：“我希望我们能继续保护开源！”
5. 第六句中的“getting a realistic grasp of AI’s risks” 翻译为“现实地理解人工智能风险”可以更准确地表达原文的意思，建议改为：“根据我与立法者的交谈，我对美国联邦政府在实际评估人工智能风险方面取得的进展感到鼓舞。”
6. 最后一句中的“guardrails are needed” 翻译为“需要设立一些规则”可以更准确地表达原文的意思，建议改为：“明确地说，需要设立一些规范。但这些规范应该针对人工智能的具体应用，而不是通用的人工智能技术。”


<h2>第三轮交互，把前两轮的结果作为输入，让llm自我微调输出最终结果<h2

In [17]:
refiner_system_prompt = f"You are an expert linguist, specializing in translation editing from English to Traditional Chinese."

refined_translate_user_prompt = f"""Your task is to carefully read, then edit, a translation from English to Simplified Chinese, taking into
account a list of expert suggestions and constructive criticisms.

The source text, the initial translation, and the expert linguist suggestions are delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT>, <TRANSLATION></TRANSLATION> and <EXPERT_SUGGESTIONS></EXPERT_SUGGESTIONS> \\
as follows:

<SOURCE_TEXT>
{source_text}
</SOURCE_TEXT>

<TRANSLATION>
{translation_1}
</TRANSLATION>

<EXPERT_SUGGESTIONS>
{reflection}
</EXPERT_SUGGESTIONS>

Please take into account the expert suggestions when editing the translation. Edit the translation by ensuring:

(i) accuracy (by correcting errors of addition, mistranslation, omission, or untranslated text),
(ii) fluency (by applying Simplified Chinese grammar, spelling and punctuation rules and ensuring there are no unnecessary repetitions), \\
(iii) style (by ensuring the translations reflect the style of the source text)
(iv) terminology (inappropriate for context, inconsistent use), or
(v) other errors.

Output only the new translation and nothing else."""

refined_user_prompt = refined_translate_user_prompt.format(
        source_text=source_text,
        translation_1=translation_1,
        reflection=reflection,
    )


In [18]:
#最终输出结果
message3 = [
    {"role":"system", "content": refiner_system_prompt},
    {"role":"user", "content": refined_user_prompt}
]

response = ask(messages=message3)

translation_2 = response.choices[0].message.content
print(translation_2)

上周，我在美国国会的一个由立法者和商界领袖参加的活动上谈到了人工智能和监管问题。我对开源社区在抵制那些可能抑制创新的法规方面取得的进展感到鼓舞。然而，反对开源的人士不断改变他们的论点，最近的担忧集中在开源对国家安全的影响上。我希望我们能继续保护开源！

根据我与立法者的交谈，我对美国联邦政府在实际评估人工智能风险方面取得的进展感到鼓舞。明确地说，需要设立一些规范。但这些规范应该针对人工智能的具体应用，而不是通用的人工智能技术。


<h2>注：Self-Refine的效果并不一定会显著提升翻译质量，但可以提升翻译的自然度、流畅度、准确性、风格和一致性，有助于提升翻译的可读性和理解度。
且不一定都是三轮交互的格式，可以根据实际情况调整交互次数和内容。<h2